Final Project, Isabel and Wyatt

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz 
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark import SparkFiles
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
#spark= SparkSession.builder.appName('aa').getOrCreate()
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import concat, unix_timestamp
from pyspark.sql import functions as F
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql.functions import split
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.types import DateType


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
peopledata = spark.read.csv('drive/My Drive/Cloud Computing/People.csv',inferSchema = True, header=True)
battingdata = spark.read.csv('drive/My Drive/Cloud Computing/Batting.csv',inferSchema = True, header=True)
fieldingdata = spark.read.csv('drive/My Drive/Cloud Computing/Fielding.csv',inferSchema = True, header=True)
fieldingOFdata = spark.read.csv('drive/My Drive/Cloud Computing/FieldingOF.csv',inferSchema = True, header=True)
pitchingdata = spark.read.csv('drive/My Drive/Cloud Computing/Pitching.csv',inferSchema = True, header=True)

In [ ]:
batting = peopledata.join(battingdata, on=['playerID'], how='inner')
fielding = peopledata.join(fieldingdata, on=['playerID'], how='inner')
fieldingOF = peopledata.join(fieldingOFdata, on=['playerID'], how='inner')
pitching =peopledata.join(pitchingdata, on=['playerID'], how='inner')

In [ ]:
batting.show(15)




+---------+---------+----------+--------+------------+----------+-------------+---------+----------+--------+------------+----------+-------------+---------+-----------+------------------+------+------+----+------+----------+----------+--------+---------+------+-----+------+----+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+
| playerID|birthYear|birthMonth|birthDay|birthCountry|birthState|    birthCity|deathYear|deathMonth|deathDay|deathCountry|deathState|    deathCity|nameFirst|   nameLast|         nameGiven|weight|height|bats|throws|     debut| finalGame| retroID|  bbrefID|yearID|stint|teamID|lgID|  G| AB|  R|  H| 2B| 3B| HR|RBI| SB| CS| BB| SO| IBB| HBP|  SH|  SF|GIDP|
+---------+---------+----------+--------+------------+----------+-------------+---------+----------+--------+------------+----------+-------------+---------+-----------+------------------+------+------+----+------+----------+----------+--------+---------+------+-----+------+----+---+---+---+

In [ ]:
list_a = batting.select("birthYear").rdd.map(lambda x: x.asDict()["birthYear"]).collect()

list_b = batting.select("yearID").rdd.map(lambda x: x.asDict()["yearID"]).collect()

result = list(set(list_b).difference(list_a))


In [ ]:
#batting['new_col'] = result
#batting = batting.assign(Age = result)
#batting['new_col'] = np.array(result)
batting.withColumn("age", batting.yearID - batting.birthYear).show()

+---------+---------+----------+--------+------------+----------+-------------+---------+----------+--------+------------+----------+-------------+---------+-----------+------------------+------+------+----+------+----------+----------+--------+---------+------+-----+------+----+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+
| playerID|birthYear|birthMonth|birthDay|birthCountry|birthState|    birthCity|deathYear|deathMonth|deathDay|deathCountry|deathState|    deathCity|nameFirst|   nameLast|         nameGiven|weight|height|bats|throws|     debut| finalGame| retroID|  bbrefID|yearID|stint|teamID|lgID|  G| AB|  R|  H| 2B| 3B| HR|RBI| SB| CS| BB| SO| IBB| HBP|  SH|  SF|GIDP| age|
+---------+---------+----------+--------+------------+----------+-------------+---------+----------+--------+------------+----------+-------------+---------+-----------+------------------+------+------+----+------+----------+----------+--------+---------+------+-----+------+----+--

In [47]:
vAssembler1 = VectorAssembler(inputCols = ["AB", "RBI", "R"], outputCol = "M1")

In [48]:
output= vAssembler1.transform(batting)

In [49]:
output.tail(4)

[Row(playerID='zobribe01', birthYear=1981, birthMonth=5, birthDay=26, birthCountry='USA', birthState='IL', birthCity='Eureka', deathYear=None, deathMonth=None, deathDay=None, deathCountry=None, deathState=None, deathCity=None, nameFirst='Ben', nameLast='Zobrist', nameGiven='Benjamin Thomas', weight=210, height=75, bats='B', throws='R', debut='2006-08-01', finalGame='2019-09-29', retroID='zobrb001', bbrefID='zobribe01', yearID=2015, stint=1, teamID='OAK', lgID='AL', G=67, AB=235, R=39, H=63, 2B=20, 3B=2, HR=6, RBI=33, SB=1, CS=1, BB=33, SO=26, IBB=2, HBP=0, SH=0, SF=3, GIDP=5, M1=DenseVector([235.0, 33.0, 39.0])),
 Row(playerID='zobribe01', birthYear=1981, birthMonth=5, birthDay=26, birthCountry='USA', birthState='IL', birthCity='Eureka', deathYear=None, deathMonth=None, deathDay=None, deathCountry=None, deathState=None, deathCity=None, nameFirst='Ben', nameLast='Zobrist', nameGiven='Benjamin Thomas', weight=210, height=75, bats='B', throws='R', debut='2006-08-01', finalGame='2019-09-29

In [33]:
data_split = output.randomSplit([0.7, 0.3])
training_data = data_split[0]
test_data = data_split[1]

In [38]:
data_new = output.sample
data_new

In [39]:
print(data_new)



<bound method DataFrame.sample of DataFrame[playerID: string, birthYear: int, birthMonth: int, birthDay: int, birthCountry: string, birthState: string, birthCity: string, deathYear: int, deathMonth: int, deathDay: int, deathCountry: string, deathState: string, deathCity: string, nameFirst: string, nameLast: string, nameGiven: string, weight: int, height: int, bats: string, throws: string, debut: string, finalGame: string, retroID: string, bbrefID: string, yearID: int, stint: int, teamID: string, lgID: string, G: int, AB: int, R: int, H: int, 2B: int, 3B: int, HR: int, RBI: int, SB: int, CS: int, BB: int, SO: int, IBB: int, HBP: int, SH: int, SF: int, GIDP: int, _M1: vector]>


In [51]:
M1 = LinearRegression(featuresCol = "M1", labelCol="H").fit(batting)

# print("Coefficients: " + str(M1.coefficients))
# print("Intercept: " + str(M1.intercept))

IllegalArgumentException: ignored